# Building Model

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda


In [43]:
inputs = pd.read_csv('inputs.csv')

/tmp/ipykernel_41907/62274928.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  inputs = pd.read_csv('inputs.csv')


In [44]:
inputs = inputs.drop(columns=['Unnamed: 0.1','Unnamed: 0'])    


In [45]:
inputs.columns

Index(['DCA License Number', 'License Type', 'License Expiration Date',
       'License Creation Date', 'Industry', 'Business Name',
       'Address Building', 'Address ZIP', 'Address Borough', 'BIN', 'Latitude',
       'Longitude', 'zipcode', 'city', 'state', 'lat', 'lng', 'population',
       'density', 'land_area', 'water_area', 'zip', 'hour 0', 'hour 1',
       'hour 2', 'hour 3', 'hour 4', 'hour 5', 'hour 6', 'hour 7', 'hour 8',
       'hour 9', 'hour 10', 'hour 11', 'hour 12', 'hour 13', 'hour 14',
       'hour 15', 'hour 16', 'hour 17', 'hour 18', 'hour 19', 'hour 20',
       'hour 21', 'hour 22', 'hour 23', 'total counts'],
      dtype='object')

In [46]:
inputs['zipcode']

0         11214.0
1         10028.0
2         10023.0
3         10460.0
4         10454.0
           ...   
182061    11235.0
182062    11231.0
182063    10001.0
182064    10301.0
182065    11231.0
Name: zipcode, Length: 182066, dtype: float64

In [6]:
y_set = pd.DataFrame(inputs['zipcode'])
x_set = inputs[['Industry','lng', 'lat', 'hour 0', 'hour 1', 'hour 2', 'hour 3', 'hour 4', 'hour 5', 'hour 6', 'hour 7', 'hour 8', 'hour 9', 'hour 10', 'hour 11', 'hour 12', 'hour 12', 'hour 13', 'hour 14', 'hour 15', 'hour 16', 'hour 17', 'hour 18', 'hour 19', 'hour 20', 'hour 21', 'hour 22', 'hour 23']]

In [7]:
x_set.shape

(182066, 28)

In [8]:
from sklearn.preprocessing import OneHotEncoder
# Encoding the 'Industry' column in x_set
encoder = OneHotEncoder(sparse_output=False)
industry_encoded = encoder.fit_transform(x_set[['Industry']])
industry_encoded_df = pd.DataFrame(industry_encoded, columns=encoder.get_feature_names_out(['Industry']))

# Combine encoded industry with the rest of x_set
x_set = pd.concat([x_set.reset_index(drop=True), industry_encoded_df], axis=1).drop(columns=['Industry'])

# Encode y_set if needed (example for binary classification)
y_encoder = OneHotEncoder(sparse_output=False)
y_set_encoded = y_encoder.fit_transform(y_set)
y_set_encoded = pd.DataFrame(y_set_encoded)

# Check shapes and memory usage
print("Shape of x_set:", x_set.shape)
print("Shape of y_set_encoded:", y_set_encoded.shape)
print("Memory usage of x_set (MB):", x_set.memory_usage(deep=True).sum() / 1024**2)
print("Memory usage of y_set_encoded (MB):", y_set_encoded.memory_usage(deep=True).sum() / 1024**2)
print("Data types in x_set:\n", x_set.dtypes)
print("Data types in y_set_encoded:\n", y_set_encoded.dtypes)


Shape of x_set: (182066, 88)
Shape of y_set_encoded: (182066, 116)
Memory usage of x_set (MB): 122.23682022094727
Memory usage of y_set_encoded (MB): 161.13031387329102
Data types in x_set:
 lng                                float64
lat                                float64
hour 0                             float64
hour 1                             float64
hour 2                             float64
                                    ...   
Industry_Ticket Seller Business    float64
Industry_Tobacco Retail Dealer     float64
Industry_Tow Truck Company         float64
Industry_Tow Truck Driver          float64
Industry_Tow Truck Exemption       float64
Length: 88, dtype: object
Data types in y_set_encoded:
 0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
111    float64
112    float64
113    float64
114    float64
115    float64
Length: 116, dtype: object


In [9]:
from torch.utils.data import TensorDataset, DataLoader
x_set_tensor = torch.tensor(x_set.values, dtype=torch.float32)
y_set_tensor = torch.tensor(y_set_encoded.values, dtype=torch.float32)

# Create dataset and dataloader
dataset = TensorDataset(x_set_tensor, y_set_tensor)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Print shapes to confirm
print("Shape of x_set_tensor:", x_set_tensor.shape)
print("Shape of y_set_tensor:", y_set_tensor.shape)

Shape of x_set_tensor: torch.Size([182066, 88])
Shape of y_set_tensor: torch.Size([182066, 116])


In [10]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_set_tensor, y_set_tensor, test_size=0.2, random_state=42)

# Create DataLoaders for training and testing sets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False)


In [32]:
import torch
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size=88, hidden_size=5, num_layers=5, output_size=116):
        super(SimpleNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        # Define the input layer
        self.input_layer = nn.Linear(input_size, hidden_size)
        
        # Define the hidden layers
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_size, hidden_size) for _ in range(num_layers - 1)])
        
        # Define the output layer
        self.output_layer = nn.Linear(hidden_size, output_size)
        
        # Activation function
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # Pass through input layer
        x = self.relu(self.input_layer(x))
        
        # Pass through hidden layers
        for layer in self.hidden_layers:
            x = self.relu(layer(x))
        
        # Pass through output layer
        x = self.output_layer(x)
        return x

# Example usage
model = SimpleNN()
print(model)


SimpleNN(
  (input_layer): Linear(in_features=88, out_features=5, bias=True)
  (hidden_layers): ModuleList(
    (0-3): 4 x Linear(in_features=5, out_features=5, bias=True)
  )
  (output_layer): Linear(in_features=5, out_features=116, bias=True)
  (relu): ReLU()
)


In [33]:
model = SimpleNN().to(device)

# Check where the model is located
print(f"Model is on device: {next(model.parameters()).device}")


Model is on device: cuda:0


In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [36]:
from tqdm import tqdm
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    train_loss = 0
    
    # Create a progress bar for the training loop
    train_progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for inputs, targets in train_progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
        # Update progress bar description
        train_progress_bar.set_postfix(loss=train_loss/len(train_dataloader))
    
    # Evaluation loop with progress bar
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct_predictions = 0
    total_predictions = 0

Epoch 2/20:  72%|███████▏  | 10456/14566 [56:41<22:16,  3.07it/s, loss=3.33] 


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [29]:
# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
test_loss = 0
correct_predictions = 0
total_predictions = 0

test_loss = 0
with torch.no_grad():  # Disable gradient calculation for evaluation
    for inputs, targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        _, actual = torch.max(targets, 1)
        correct_predictions += (predicted == actual).sum().item()
        total_predictions += targets.size(0)

test_loss /= len(test_dataloader)  # Calculate the average loss
accuracy = correct_predictions / total_predictions  # Calculate accuracy

print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

Test Loss: 4.6404
Test Accuracy: 0.0201
